In [1]:
import networkx as nx
# import urllib -- Did not work on my linux machine - using requests instead
import re
import tqdm
import collections
import requests
#import community.community_louvain as cl
import json
import matplotlib.pyplot as plt
import pandas as pd
import copy
import re

In [39]:
with open('origin_dict.txt') as f: 
    data = f.read() 
      
# reconstructing the data as a dictionary 
js = json.loads(data)
len(js.keys())

711

In [40]:
dates_dict = dict().fromkeys(js.keys())

In [63]:
for x in dates_dict.keys():
    # dates_dict[x]
    if len(js[x][0]) > 0:
        if not js[x][0][0].isalpha(): # if there is at least one number
            dates_dict[x] = js[x][0][0]
        elif len(js[x][0])> 1 and not js[x][0][1].isalpha(): # if there is at least one number
            dates_dict[x] = js[x][0][1]

for k, v in dates_dict.items(): 
    if v:
        new = re.findall('[0-9]+', v)
        if len(new)>0:
            dates_dict[k] = new[0]

clean_dict_dates = {}
for k, v in dates_dict.items():
    if v:
        if len(v) == 2:
            clean_dict_dates[k] = v + '00'       
        else: clean_dict_dates[k] = v   

# known issues: 
clean_dict_dates['Hip hop music'] = '1973'
clean_dict_dates['East Coast hip hop'] = '1973'
clean_dict_dates['Diva house'] = '1990'
clean_dict_dates['V-pop'] = '1980'

In [5]:
places_dict = dict().fromkeys(js.keys())

for x in places_dict.keys():
    # dates_dict[x]
    if len(js[x][1]) > 0:
        places_dict[x] = js[x][1]

In [8]:
cleaner_dict = {}

blacklist =  ['Circa', 'circa', '']

for k, v in places_dict.items(): 
    if v: 
        if len(v)>1:
            places = []
            for p in v: 
                to_add = re.sub('[\[\]{}]+', '', p)
                if to_add not in blacklist:
                    places.append(to_add)
    
            cleaner_dict[k] = places[0] # adding only first one 
        if len(v) == 1: 
            to_add = re.sub('[\[\]{}]+', '', v[0])
            if to_add not in blacklist:
                cleaner_dict[k] = to_add

In [9]:
import geopy

In [10]:
# importing geopy library
from geopy.geocoders import Nominatim
 
# calling the Nominatim tool
loc = Nominatim(user_agent="GetLoc")


In [30]:
correction_dict = {
'Korea':'People republic of Korea',
'US':'United States',
'UK':'United Kingdom',
'U.S.':'United States',
'U.K.':'United Kingdom',
'Mid-':'California',
'Cajun':'Louisiana',
'Sweden':'Stockholm',
'Europe':' Suchowola',
'Egypt':'Kairo',
'Russia':'Moskow',
'Ibiza small': 'Ibiza',
'Germany':'Berlin',
'Greece':'Athens',
'Scandinavia':'Copenhagen',
'Yugoslavia':'Serbia',
'Southern Europe':'Venice',
'Western Europe':'Lyon',
'Northern Europe':'Oslo',
'Oranie': 'Algeria',
'Mid':'California',
'Cubans':'Cuba',
'Northeastern Mexico':'Mexico',
'United&nbsp;States':'United States',
'Southeastern Brazil':'Brasil',
'West Coast of the United States':'San Francisco',
'United&nbsp;States':'United States',
'Pacific Northwest':'Portland',
'United States and Sweden':'United States',
'Western Nigeria':'Nigeria',
'Igboland':'Nigeria',
'Principality of Serbia':'Serbia'
}

In [31]:
import tqdm

In [33]:
lat_long_dict_2 = {}
for k, v in tqdm.tqdm(cleaner_dict.items(), total=len(cleaner_dict.keys())):
    # entering the location name
    try:
        if v in correction_dict.keys():
            getLoc = loc.geocode(correction_dict[v])
        else:
            getLoc = loc.geocode(v)
        lat_long_dict_2[k] = [getLoc.latitude, getLoc.longitude]
    except: 
        print('failed for: ' + v)

  1%|██▏                                                                                                                                                                                               | 7/618 [00:03<04:38,  2.19it/s]

failed for: Mid-twentieth-


 41%|█████████████████████████████████████████████████████████████████████████████▉                                                                                                                  | 251/618 [02:12<02:49,  2.16it/s]

failed for: nowrap


 53%|█████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                          | 326/618 [02:51<02:14,  2.17it/s]

failed for: nowrap


 57%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                   | 351/618 [03:05<02:04,  2.14it/s]

failed for: AMearly


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 618/618 [05:30<00:00,  1.87it/s]


In [89]:
with open('long_lat_dict.json', 'w') as f:
    json.dump(lat_long_dict_2, f)

In [64]:
with open('dates_dict.json', 'w') as f:
    json.dump(clean_dict_dates, f)

In [87]:
getLoc = loc.geocode('Brno', language='English')

In [88]:
getLoc.raw

{'place_id': 165957977,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 438171,
 'lat': '49.1922443',
 'lon': '16.6113382',
 'class': 'boundary',
 'type': 'administrative',
 'place_rank': 16,
 'importance': 0.644589076488367,
 'addresstype': 'city',
 'name': 'Brno',
 'display_name': 'Brno, okres Brno-město, Jihomoravský kraj, Česko',
 'boundingbox': ['49.1098796', '49.2943708', '16.4279898', '16.7278346']}

In [83]:
country_dict = {}
for k, v in tqdm.tqdm(cleaner_dict.items(), total=len(cleaner_dict.keys())):
    # entering the location name
    try:
        if v in correction_dict.keys():
            getLoc = loc.geocode(correction_dict[v])
        else:
            getLoc = loc.geocode(v)
        country_dict[k] = getLoc.raw['display_name'].split(',')[-1]
    except: 
        print('failed for: ' + v)

  1%|██▏                                                                                                                                                                                               | 7/618 [00:03<04:45,  2.14it/s]

failed for: Mid-twentieth-


 41%|█████████████████████████████████████████████████████████████████████████████▉                                                                                                                  | 251/618 [02:14<03:37,  1.69it/s]

failed for: nowrap


 53%|█████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                          | 326/618 [02:53<02:18,  2.11it/s]

failed for: nowrap


 57%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                   | 351/618 [03:07<02:13,  2.01it/s]

failed for: AMearly


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 618/618 [05:30<00:00,  1.87it/s]


In [84]:
with open('country_dict.json', 'w') as f:
    json.dump(country_dict, f)